In [ ]:
import itertools
import copy

variables = dict()
functions = dict()
reverse_assignments = dict()

reading_first_part = True

x_bit_length = 0
y_bit_length = 0

with open('input') as f:
    for y, line in enumerate(f):
        line = line.strip()
        if len(line) == 0:
            reading_first_part = False
            continue
        if reading_first_part:
            line = line.split(": ")
            variables[line[0]] = int(line[1])
            if line[0][0] == 'x':
                x_bit_length += 1
            if line[0][0] == 'y':
                y_bit_length += 1
        if reading_first_part is False:
            line = line.split(" ")
            functions[(line[0], line[1], line[2])] = line[4]
            reverse_assignments[line[4]] = [line[0], line[1], line[2]]

max_x = sum([2**x for x in range(x_bit_length)])
max_y = sum([2**y for y in range(y_bit_length)])

original_variables = set([key for key in variables])

def run_add(functions, variables):
    proceed = True
    evaluated_functions = set()
    while proceed:
        proceed = False
        for function in functions:
            if function in evaluated_functions:
                continue
            if function[0] in variables and function[2] in variables:
                operation = function[1]
                destination = functions[function]
                evaluated_functions.add(function)
                if operation == "AND":
                    result = variables[function[0]] & variables[function[2]]
                elif operation == "OR":
                    result = variables[function[0]] | variables[function[2]]
                elif operation == "XOR":
                    result = variables[function[0]] ^ variables[function[2]]
                variables[destination] = result
                proceed = True

    final_z = 0
    for variable in variables:
        if variable[0] == 'z':
            base = int(variable[1:])
            final_z += variables[variable] * 2**base
    return final_z

x = 0
for variable in variables:
    if variable[0] == 'x':
        base = int(variable[1:])
        x += variables[variable] * 2**base
x_bin = str(bin(x))[2:].zfill(x_bit_length)[::-1]

y = 0
for variable in variables:
    if variable[0] == 'y':
        base = int(variable[1:])
        y += variables[variable] * 2**base
y_bin = str(bin(y))[2:].zfill(y_bit_length)[::-1]

initial_variables = copy.deepcopy(variables)

z = run_add(functions, variables)
expected_z = x+y
z_diff = z^expected_z
diff_string = str(bin(z_diff))
diff_string_cuted = diff_string[2:]
diff_rotated = diff_string_cuted[::-1]

# # Problem space

# problem_space = set()
# for i in range(len(diff_rotated)):
#     if diff_rotated[i] == '1':
#         set_number = 'z'+str(i).zfill(2)
#         add_nodes = [set_number]
#         while len(add_nodes) > 0:
#             next_nodes = []
#             for node in add_nodes:
#                 if node not in reverse_assignments:
#                     continue
#                 problem_space.add(tuple(reverse_assignments[node]))
#                 next_nodes.append(reverse_assignments[node][0])
#                 next_nodes.append(reverse_assignments[node][2])
#             add_nodes = next_nodes

# print("Problem space", problem_space)
# print(len(problem_space), len(functions))

# cutting gates with unchanged outputs

pair_node = dict()
next_function = dict()

for i in range(len(diff_rotated)):
    if diff_rotated[i] == '1':
        set_number = 'z'+str(i).zfill(2)
        add_nodes = [reverse_assignments[set_number]]
        while len(add_nodes) > 0:
            next_nodes = []
            for node in add_nodes:
                if node[0] not in reverse_assignments and node[2] not in reverse_assignments:
                    continue
                up_node = node[0]
                down_node = node[2]
                pair_node[tuple(reverse_assignments[up_node])] = reverse_assignments[down_node]
                pair_node[tuple(reverse_assignments[down_node])] = reverse_assignments[up_node]
                next_function[tuple(reverse_assignments[up_node])] = tuple(node)
                next_function[tuple(reverse_assignments[down_node])] = tuple(node)
                next_nodes.append(reverse_assignments[up_node])
                next_nodes.append(reverse_assignments[down_node])
            add_nodes = next_nodes


checks = 0
found = False
for pair1 in itertools.combinations(functions, 2):
    if variables[functions[pair1[0]]] == variables[functions[pair1[1]]]:
        continue

    if pair1[0] in next_function and pair1[1] in next_function:
        next_f_1 = next_function[pair1[0]]
        operation_1 = next_f_1[1]
        expected_output_1 = variables[functions[next_f_1]]

        next_f_2 = next_function[pair1[1]]
        operation_2 = next_f_2[1]
        expected_output_2 = variables[functions[next_f_2]]

        up_val = variables[next_f_1[0]]
        down_val = variables[functions[pair1[1]]]
        if operation_1 == "AND":
            result = up_val & down_val
        elif operation_1 == "OR":
            result = up_val | down_val
        elif operation_1 == "XOR":
            result = up_val ^ down_val
        if result == expected_output_1:
            continue

        up_val = variables[functions[pair1[0]]]
        down_val = variables[next_f_2[2]]
        if operation_2 == "AND":
            result = up_val & down_val
        elif operation_2 == "OR":
            result = up_val | down_val
        elif operation_2 == "XOR":
            result = up_val ^ down_val
        if result == expected_output_2:
            continue
    
    # checks += 1

    for pair2 in itertools.combinations(functions, 2):
        if variables[functions[pair2[0]]] == variables[functions[pair2[1]]]:
            continue

        if pair2[0] in next_function and pair2[1] in next_function:
            compared_function_1 = next_function[pair2[0]]
            operation_1 = compared_function_1[1]
            expected_output_1 = variables[functions[compared_function_1]]

            compared_function_2 = next_function[pair2[1]]
            operation_2 = compared_function_2[1]
            expected_output_2 = variables[functions[compared_function_2]]

            up_val = variables[compared_function_1[0]]
            down_val = variables[functions[pair2[1]]]
            if operation_1 == "AND":
                result = up_val & down_val
            elif operation_1 == "OR":
                result = up_val | down_val
            elif operation_1 == "XOR":
                result = up_val ^ down_val
            if result == expected_output_1:
                continue

            up_val = variables[functions[pair2[0]]]
            down_val = variables[compared_function_2[2]]
            if operation_2 == "AND":
                result = up_val & down_val
            elif operation_2 == "OR":
                result = up_val | down_val
            elif operation_2 == "XOR":
                result = up_val ^ down_val
            if result == expected_output_2:
                continue

        if pair2[0] in pair1 or pair2[1] in pair1 or pair1[0] in pair2 or pair1[1] in pair2:
            continue

        checks += 1
        # if checks % 10000 == 0:
        #     print(checks/9054368)

        # temp_variables = initial_variables.copy()
        # temp_variables = copy.copy(initial_variables)

        # functions[pair1[0]], functions[pair1[1]] = functions[pair1[1]], functions[pair1[0]]
        # functions[pair2[0]], functions[pair2[1]] = functions[pair2[1]], functions[pair2[0]]

        # number = run_add(functions, temp_variables)
                
        # if number == x+y:
        #     print(number, x, y)
        #     print("Znalezione")
        #     print(pair1, pair2)
        #     found = True
        
        # functions[pair1[0]], functions[pair1[1]] = functions[pair1[1]], functions[pair1[0]]
        # functions[pair2[0]], functions[pair2[1]] = functions[pair2[1]], functions[pair2[0]]

    #     if found:
    #         break
    # if found:
    #     break

print("Checks", checks)


In [ ]:
210304263
206523570
104393421
102498788
71994610